# Preparamos variables y funciones para la obtencion de datos

## Cargamos las variables de entorno necesarias (API KEYS)

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Creamos funciones necesarias (JsonFormatter, FetchData, MatchCategories, etc)

### Creamos la funcion Regex para poder buscar palabras clave en las respuestas de las APIS

In [2]:
import re

# Las categorias a buscar
CATEGORIES = ["memes", "game", "gaming", "ai", "real world assets"]

# Crear una expresión regular que coincida con cualquier palabra clave
regex_pattern = r"\b(?:{})\b".format("|".join(CATEGORIES))
regex = re.compile(regex_pattern, re.IGNORECASE)

### Creamos la funcion pretty_print para poder imprimir los datos de una forma mas legible


In [3]:
import json

def pretty_print(data):
    print(json.dumps(data, indent=2))

### Creamos la funcion y variables para poder realizar las peticiones a la APIS

In [4]:
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from enum import Enum


# Enum para las APIs
class APIS(Enum):
    COINMARKET = {
        "url": "https://pro-api.coinmarketcap.com/v1/",
        "headers": {
            "Accepts": "application/json",
            "X-CMC_PRO_API_KEY": os.environ.get("API_KEY_COINMARKET"),
        },
    }
    MESSARI = {
        "url": "https://api.messari.io/",
        "headers": {
            "accept": "application/json",
            "x-messari-api-key": os.environ.get("API_KEY_MESSARI"),
        },
    }

In [5]:
# Función para obtener datos de una API
# api = Usar Enum APIS
# uri = uri de la API
# parameters = parámetros de la API
def fetch_data(api: APIS, uri: str, parameters: dict) -> dict:
    api_ = api.value
    session = Session()
    session.headers.update(api_["headers"])
    try:
        response = session.get(api_["url"] + uri, params=parameters)
        return json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        return {"error": e}

### Crear funcion para guardar archivos

In [6]:
# Función para guardar los datos en un archivo JSON
def save_json(FILE: dict | list[dict], dir: str):
    dirs = dir.split("/")[:-1]
    if dirs:
        # Crear un directorio para guardar los metadatos
        os.makedirs("/".join(dirs), exist_ok=True)

    # Guardamos las categorias filtradas
    with open(dir, "w") as file:
        json.dump(FILE, file)

# Using CoinMarketAPI


## Usamos la documentacion para obtener las categorias de Coin Market API


In [7]:
# Obtenemos la lista de criptomonedas
categories_coinmarket = fetch_data(
    api=APIS.COINMARKET, uri="cryptocurrency/categories", parameters={}
)
# Obtenemos la data de las categorias
categories_coinmarket = categories_coinmarket["data"]

In [8]:
# Filtrar las categorías utilizando la expresión regular
categories_coinmarket = list(
    filter(lambda categoria: regex.search(
        categoria["title"].lower()), categories_coinmarket)
)

In [9]:
save_json(categories_coinmarket, "metadata/categories_coinmarket.json")

## Ahora que tenemos los ids de las categorias que necesitamos (IA, Games, RWA, etc), obtenemos la data de las criptomonedas

In [18]:
cryptos_coinmarket = {}
for category in categories_coinmarket:
    cryptos = fetch_data(
        api=APIS.COINMARKET,
        uri="cryptocurrency/category",
        parameters={"id": category["id"]},
    )
    cryptos_coinmarket[category["title"]] = {"coins": cryptos["data"]["coins"]}

In [27]:
save_json(cryptos_coinmarket, "data/cryptos_coinmarket.json")

# Using Messari


## Usamos la documentacion para obtener las categorias de Messari API